**SetUp**

In [ ]:
!pip install compressai --quiet
!pip install -q gdown
!pip install diffusers transformers accelerate --quiet

In [ ]:
import os
import torch
from PIL import Image, ImageEnhance, ImageFilter
from torchvision import transforms
#from compressai.zoo import cheng2020_anchor
from tqdm import tqdm
from google.colab import files
import zipfile
import random
import torchvision.transforms.functional as TF
import io
import numpy as np
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.transforms.functional import to_pil_image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import shutil
from glob import glob


src_folder = '/content/drive/MyDrive/watermarked/VineSem'
dst_folder = "/content/data"


os.makedirs(dst_folder, exist_ok=True)

image_extensions = ("*.jpg", "*.jpeg", "*.png")
image_files = []
for ext in image_extensions:
    image_files.extend(glob(os.path.join(src_folder, ext)))


for file in image_files:
    shutil.copy(file, dst_folder)

print(f"✅ Copied {len(image_files)} images to {dst_folder}")


In [ ]:
src_folder = "/content/data"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

**AICompress**

In [ ]:
from compressai.zoo import cheng2020_anchor
def compute_padding(h_w, factor=64):
    h, w = h_w
    pad_h = (factor - h % factor) % factor
    pad_w = (factor - w % factor) % factor
    return (0, pad_w, 0, pad_h)


output_folder = "/content/drive/MyDrive/edits/1inoutputs/AICompress"
os.makedirs(output_folder, exist_ok=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = cheng2020_anchor(quality=4, pretrained=True).eval().to(device)

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])


image_files = sorted([
    f for f in os.listdir(src_folder)
    if f.lower().endswith((".jpg", ".jpeg", ".png"))
])

for fname in tqdm(image_files, desc="Compressing+Decompressing"):
    try:
        img_path = os.path.join(src_folder, fname)
        img = Image.open(img_path).convert("RGB")
        x = transform(img).unsqueeze(0).to(device)


        pad = compute_padding(x.shape[-2:], factor=64)
        x_padded = torch.nn.functional.pad(x, pad, mode="constant", value=0)


        with torch.no_grad():
            out_enc = model.compress(x_padded)
            out_dec = model.decompress(out_enc["strings"], out_enc["shape"])
            x_hat = out_dec["x_hat"]

        x_hat = x_hat[:, :, :x.shape[2], :x.shape[3]].squeeze().clamp(0, 1).cpu()
        output_img = transforms.ToPILImage()(x_hat)

        output_img.save(os.path.join(output_folder, fname))

    except Exception as e:
        print(f"❌ Error processing {fname}: {e}")

**Colour/hue distortions**

In [ ]:
dst_folder = "/content/drive/MyDrive/edits/1inoutputs/common_colour"
os.makedirs(dst_folder, exist_ok=True)

param_ranges = {
    "brightness": (0.8, 1.8),
    "contrast": (0.8, 1.8),
    "saturation": (0.8, 1.5),
    "hue_shift": (-0.1, 0.1),
    "white_balance": (0.9, 1.1),
    "exposure": (0.9, 1.1),
}

def apply_color_manipulations(img):
    ops = ["brightness", "contrast", "saturation", "hue_shift", "white_balance", "exposure"]
    random.shuffle(ops)

    for op in ops:
        if op == "brightness":
            factor = random.uniform(*param_ranges["brightness"])
            img = ImageEnhance.Brightness(img).enhance(factor)

        elif op == "contrast":
            factor = random.uniform(*param_ranges["contrast"])
            img = ImageEnhance.Contrast(img).enhance(factor)

        elif op == "saturation":
            factor = random.uniform(*param_ranges["saturation"])
            img = ImageEnhance.Color(img).enhance(factor)

        elif op == "hue_shift":
            hue_factor = random.uniform(*param_ranges["hue_shift"])
            img = TF.adjust_hue(img, hue_factor)

        elif op == "white_balance":
            factor = random.uniform(*param_ranges["white_balance"])
            r, g, b = img.split()
            r = r.point(lambda x: min(255, x * factor))
            b = b.point(lambda x: min(255, x / factor))
            img = Image.merge("RGB", (r, g, b))

        elif op == "exposure":
            factor = random.uniform(*param_ranges["exposure"])
            img = TF.adjust_gamma(img, gamma=1.0/factor)

    return img

for fname in sorted(os.listdir(src_folder)):
    if not fname.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    try:
        img_path = os.path.join(src_folder, fname)
        img = Image.open(img_path).convert("RGB")
        img_out = apply_color_manipulations(img)
        img_out.save(os.path.join(dst_folder, fname))
        print(f"✅ Processed: {fname}")

    except Exception as e:
        print(f"❌ Failed on {fname}: {e}")


**Lossy/compression distortions**

In [ ]:
dst_folder = "/content/drive/MyDrive/edits/1inoutputs/common_comp"
os.makedirs(dst_folder, exist_ok=True)



def apply_jpeg_compression(img, quality=25):
    buffer = io.BytesIO()
    img.save(buffer, format="JPEG", quality=quality)
    return Image.open(buffer)

def apply_webp_conversion(img):
    buffer = io.BytesIO()
    img.save(buffer, format="WEBP", quality=30)
    return Image.open(buffer)

def apply_downscale_upscale(img, factor=4):
    w, h = img.size
    small = img.resize((w // factor, h // factor), Image.BILINEAR)
    return small.resize((w, h), Image.BILINEAR)

def apply_bitdepth_reduction(img, bits=4):
    arr = np.array(img)
    levels = 2 ** bits
    scale = 255 // (levels - 1)
    arr = (arr // scale) * scale
    return Image.fromarray(arr.astype(np.uint8))

# === List of distortion functions in order ===
manipulations = [
    apply_jpeg_compression,
    apply_webp_conversion,
    apply_downscale_upscale,
    apply_bitdepth_reduction
]

img_files = sorted([f for f in os.listdir(src_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
for idx, fname in enumerate(img_files):
    try:
        img_path = os.path.join(src_folder, fname)
        img = Image.open(img_path).convert("RGB")

        fn = manipulations[idx % len(manipulations)]
        img_out = fn(img)
        img_out.save(os.path.join(dst_folder, fname))

        print(f"✅ Applied {fn.__name__} to {fname}")

    except Exception as e:
        print(f"❌ Failed on {fname}: {e}")

**Blur distortions**

In [ ]:
dst_folder = "/content/drive/MyDrive/edits/1inoutputs/common_blurs"
os.makedirs(dst_folder, exist_ok=True)



def apply_gaussian_blur(img, radius=3):
    return img.filter(ImageFilter.GaussianBlur(radius=radius))

def apply_motion_blur(img, size=7):
    return img.filter(ImageFilter.BoxBlur(radius=size // 2))

def apply_zoom_blur(img, factor=1.05, blend=0.5):
    w, h = img.size
    zoomed = img.resize((int(w * factor), int(h * factor)), Image.BILINEAR)
    left = (zoomed.width - w) // 2
    top = (zoomed.height - h) // 2
    zoomed = zoomed.crop((left, top, left + w, top + h))
    return Image.blend(img, zoomed, alpha=blend)

blur_manipulations = [
    apply_gaussian_blur,
    apply_motion_blur,
    apply_zoom_blur
]

img_files = sorted([f for f in os.listdir(src_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])

for idx, fname in enumerate(img_files):
    try:
        img_path = os.path.join(src_folder, fname)
        img = Image.open(img_path).convert("RGB")

        fn = blur_manipulations[idx % len(blur_manipulations)]
        img_out = fn(img)

        img_out.save(os.path.join(dst_folder, fname))
        print(f"✅ Applied {fn.__name__} to {fname}")

    except Exception as e:
        print(f"❌ Failed on {fname}: {e}")

**Noise Distortions**

In [ ]:

dst_folder = "/content/drive/MyDrive/edits/1inoutputs/common_noise"
os.makedirs(dst_folder, exist_ok=True)


def apply_gaussian_noise(img, std=0.05):
    t = TF.to_tensor(img)
    noise = torch.randn_like(t) * std
    return TF.to_pil_image((t + noise).clamp(0, 1))

def apply_speckle_noise(img, std=0.2):
    t = TF.to_tensor(img)
    noise = t * torch.randn_like(t) * std
    return TF.to_pil_image((t + noise).clamp(0, 1))

def apply_impulse_noise(img, amount=0.02):
    arr = np.array(img)
    h, w, c = arr.shape
    num_salt = np.ceil(amount * h * w * 0.5).astype(int)
    num_pepper = np.ceil(amount * h * w * 0.5).astype(int)

    coords = [np.random.randint(0, i - 1, num_salt) for i in (h, w)]
    arr[coords[0], coords[1]] = 255

    coords = [np.random.randint(0, i - 1, num_pepper) for i in (h, w)]
    arr[coords[0], coords[1]] = 0

    return Image.fromarray(arr.astype(np.uint8))

def apply_banding_artifacts(img, bands=8):
    arr = np.array(img)
    arr = (arr // (256 // bands)) * (256 // bands)
    return Image.fromarray(arr.astype(np.uint8))

noise_manipulations = [
    apply_gaussian_noise,
    apply_speckle_noise,
    apply_impulse_noise,
    apply_banding_artifacts
]

img_files = sorted([f for f in os.listdir(src_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])

for idx, fname in enumerate(img_files):
    try:
        img_path = os.path.join(src_folder, fname)
        img = Image.open(img_path).convert("RGB")

        fn = noise_manipulations[idx % len(noise_manipulations)]
        img_out = fn(img)

        img_out.save(os.path.join(dst_folder, fname))
        print(f"✅ Applied {fn.__name__} to {fname}")

    except Exception as e:
        print(f"❌ Failed on {fname}: {e}")